In [1]:
import pandas as pd
import PyPDF2
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')
pd.set_option('display.max_columns', 100)

In [2]:
pdf_file = open('data/00-391.pdf',"rb")
read_pdf = PyPDF2.PdfFileReader(pdf_file)

number_of_pages = read_pdf.getNumPages()
page = read_pdf.getPage(36)
page_content = page.extractText()
print(page_content)

                       1    modified.
            2              Lastly, I would just like to point out, at pages
            3    31 and 32 of respondent's brief, they make a statement
            4    that the danger arises to the officer at the moment that
            5    contact is made with the arrestee.  Well, in this case, as
            6    in many cases that we've cited, that danger arises
            7    regardless of the reason.  Therefore, if the arrestee
            8    voluntarily exits, it makes no -- there is no
            9    constitutional distinction.
           10              Thank you.
           11              CHIEF JUSTICE REHNQUIST:  Thank you, Mr. Krauss.
           12              The case is submitted.
           13              (Whereupon, at 11:53 a.m., the case in the
           14    above-entitled matter was submitted.)
           15
           16
           17
           18
           19
           20
           21
           22
           23
  

In [3]:
type(page_content)

str

In [4]:
len(page_content)

1390

In [5]:
dk =[]
number_of_pages = read_pdf.getNumPages()
for i in range(0,number_of_pages):
    page = read_pdf.getPage(i)
    page_content = page.extractText()
    dk.append(page_content)

In [6]:
dk.append(page_content)

In [7]:
dk

['                       1             IN THE SUPREME COURT OF THE UNITED STATES\n            2    - - - - - - - - - - - - - - - -X\n            3    FLORIDA,                       :\n            4              Petitioner           :\n            5         v.                        :  No. 00-391\n            6    ROBERT A. THOMAS               :\n            7    - - - - - - - - - - - - - - - -X\n            8                                 Washington, D.C.\n            9                                 Wednesday, April 25, 2001\n           10              The above-entitled matter came on for oral\n           11    argument before the Supreme Court of the United States at\n           12    11:14 a.m.\n           13    APPEARANCES:\n           14    ROBERT J. KRAUSS, ESQ., Senior Assistant Attorney\n           15         General, Tampa, Florida; on behalf of the Petitioner.\n           16    GREGORY G. GARRE, ESQ., Assistant to the Solicitor\n           17         General, Department 

In [8]:
#Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')
#pd.set_option('display.max_columns', 100)

In [9]:
Cases[Cases['dateArgument']=='3/26/2012']

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes
8441,2011-077,2011-077-01,2011-077-01-01,2011-077-01-01-01,6/28/2012,1,NaN,132 S. Ct. 2566,183 L. Ed. 2d 450,2012 U.S. LEXIS 4876,2011,1704,Roberts,11-393,"NATIONAL FEDERATION OF INDEPENDENT BUSINESS, e...",3/26/2012,NaN,207,NaN,363.0,NaN,1,NaN,NaN,0.0,57.0,NaN,31.0,NaN,0.0,2.0,2.0,1.0,1.0,6.0,0.0,0.0,0.0,0.0,100120.0,10.0,2.0,0.0,1.0,4.0,1.0,110.0,NaN,111.0,111.0,1,5,4


### Need to use docket from filename xxxx.pdf to tie back case to other data

In [10]:
Cases[Cases['docket']=='17-606']

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes


In [11]:
##download each PDF.
## Access its text
##put text in dataframe with docket# and map to original Cases data


In [14]:
import os
files = os.listdir('ArgText/')
paths = []
for file in files:
    paths.append('ArgText/'+file)
len(paths)

1311

In [17]:
my_dict = {}
for index, path in enumerate(paths):
    with open(path, "rb") as f:
        dk = []
        read_pdf = PyPDF2.PdfFileReader(f)
        for i in range(0,read_pdf.getNumPages()):
            page = read_pdf.getPage(i)
            page_content = page.extractText()
            dk.append(page_content)
    my_dict[files[index].replace(".pdf", "")] = " ".join(dk)##removes pdf from key and turns value array into string

In [ ]:
#files = os.listdir('ArgText/')
#paths = []
#for file in files:
    #paths.append('ArgText/'+file)

In [ ]:
#pdf_file = open('data/00-391.pdf',"rb")
#read_pdf = PyPDF2.PdfFileReader(pdf_file)

#number_of_pages = read_pdf.getNumPages()
#page = read_pdf.getPage(36)
#page_content = page.extractText()

In [ ]:
len(my_dict)

In [20]:
my_dict['01-7574']

'1234\n5\n678\n910\n11\n12\n131415\n161718\n19\n202122\n232425 IN THE SUPREME COURT OF THE UNITED STATES•- - - - - - - - - - - - - - - -X•DAVID ALLEN SATTAZAHN, :•Petitioner :•v. : No. 01-7574•PENNSYLVANIA :•- - - - - - - - - - - - - - - -X•Washington, D.C.•Monday, November 4, 2002•The above-entitled matter came on for oral•argument before the Supreme Court of the United States at•11:06 a.m.•\nAPPEARANCES:•\nROBERT B. DUNHAM, ESQ., Assistant Federal Defender,•Philadelphia, Pennsylvania; on behalf of the Petitioner.•IVA C. DOUGHERTY, ESQ., First Assistant District Attorney,•Reading, Pennsylvania; on behalf of the Respondent.•SRI SRINIVASAN, ESQ., Assistant to the Solicitor General,•Department of Justice, Washington, D.C.; on behalf of•the United States, as amicus curiae, supporting the•Respondent.•1  1234\n5\n678\n910\n11\n12\n131415\n161718\n19\n202122\n232425 C O N T E N T S•ORAL ARGUMENT OF •ROBERT B. DUNHAM, ESQ.•On behalf of the Petitioner •ORAL ARGUMENT OF•\nIVA C. DOUGHERTY, ESQ.

In [46]:
Labels = Cases[['docket','partyWinning']]

In [40]:
Text = pd.DataFrame.from_dict(my_dict, orient="index")

In [41]:
Text = Text.rename(index=str, columns={0: "Document"})

In [ ]:
def GetMaxItem(item):        
    maks=max(item, key=lambda k: len(item[k]))
    return len(item[maks]), maks

In [ ]:
GetMaxItem(my_dict)

In [54]:
import numpy as np
import re

In [42]:
Text.head()

,Document
00-1011,1 IN THE SU...
00-1021,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...
00-1045,1 2 \n ...
00-10666,12345678910\n111213141516171819202122232425 IN...
00-1072,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...


In [ ]:
#pd.merge(Text, Labels, on='Customer_id', how='inner')

In [43]:
Text['docket'] = Text.index

In [44]:
Text.reset_index(drop=True)

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072
5,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1073
6,1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...,00-1089
7,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1167
8,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1187
9,1 IN THE SU...,00-121


In [49]:
Text = pd.merge(Text, Labels, on='docket', how='inner')

In [50]:
stop_words =['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself',
            'yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself',
            'they','them','their','theirs','themselves','what','which','who','whom','this','that',
            'these','those','am','is','are','was','were','be','been','being','have','has','had',
            'having','do','does','did','doing','a','an','the','and','but','if','or','because','as',
            'until','while','of','at','by','for','with','about','against','between','into','through',
            'during','before','after','above','below','to','from','up','down','in','out','on','off',
            'over','under','again','further','then','once','here','there','when','where','why','how',
            'all','any','both','each','few','more','most','other','some','such','no','nor','not',
            'only','own','same','so','than','too','very','s','t','can','will','just','don','should',
            'now','uses','use','using','used','one','also']

,Document,docket,partyWinning
0,1 IN THE SU...,00-1011,1.0
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021,0.0
2,1 2 \n ...,00-1045,1.0
3,12345678910\n111213141516171819202122232425 IN...,00-10666,0.0
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072,1.0


In [ ]:
def preprocess(data):
    reviews_tokens = []
    for review in data:
        review = review.lower() #Convert to lower-case words
        raw_word_tokens = re.findall(r'(?:\w+)', review,flags = re.UNICODE) #remove pontuaction
        word_tokens = [w for w in raw_word_tokens if not w in stop_words] # do not add stop words
        reviews_tokens.append(word_tokens)
    return reviews_tokens #return all tokens

In [55]:
num_words = Text['Document'].apply(lambda x: len(x.split()))
num_words_mean, num_words_std = np.mean(num_words), np.std(num_words)

num_sentences = Text['Document'].apply(lambda x: len(re.split( '~ ...' ,'~'.join(x.split('.')))))
num_sentences_mean = np.mean(num_sentences)

0       13007
1       11406
2       12422
3       10089
4        9922
5       11994
6       11536
7       12608
8       13224
9        9373
10      11166
11      12034
12      11157
13      12529
14      12382
15      11589
16      13083
17      10297
18      11412
19      11495
20      11835
21      13210
22       9633
23      11442
24      12474
25      12670
26      11745
27      12517
28      11521
29      16890
        ...  
1264    12168
1265    13391
1266    12945
1267    10473
1268     8854
1269     9432
1270    10706
1271    10151
1272    11371
1273    13424
1274    12413
1275    12990
1276     9970
1277    11772
1278    13073
1279    15086
1280    12130
1281    12292
1282    10655
1283    11091
1284    17909
1285    10120
1286    14045
1287     6822
1288    12615
1289    11112
1290    11510
1291    11808
1292    11384
1293    12481
Name: Document, Length: 1294, dtype: int64

1294